In [10]:
import pandas as pd
import os
import numpy as np
import plotly
import plotly.express as px
from config import *

from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.time import Time

pd.options.display.float_format = '{:.9f}'.format

### 3D график всех элементов

In [16]:
with open(f'res_iter_p_coords_{name_pulsar}.txt', 'r') as file:
    list_for_lines = []
    text = file.read().split('\n')
    for i in text:
        list_for_lines.append(i.split())
    x_list = []
    y_list = []
    z_list = []
    colour_list = []
    list_for_lines.pop()
    for i in list_for_lines:
        z_list.append(float(i[0]))
        if len(i[1]) <= 12:
            x_list.append(i[1])
        else: 
            x_list.append(i[1][:-6]) # ограничение на длину параметра
        if len(i[2]) <= 12:
            y_list.append(i[2])
        else: 
            y_list.append(i[2][:-6])# ограничение на длину параметра
        colour_list.append(float(i[3]))

In [17]:
df = pd.DataFrame({'ra': x_list, 'dec': y_list, 'freq': z_list, 'residuals': colour_list})
fig = px.scatter_3d(df, x='ra', y='dec', z='freq', color='residuals')
plotly.offline.plot(fig, filename=f'./res_iter_ra_dec_freq_{name_pulsar}.html')

'./res_iter_ra_dec_freq_1112.html'

### 3D график лучших значений

In [6]:
with open(f'res_iter_p_coords_{name_pulsar}.txt', 'r') as file:
    list_for_lines = []
    text = file.read().split('\n')
    for i in text:
        list_for_lines.append(i.split())
    x_list = []
    y_list = []
    z_list = []
    colour_list = []
    #residuals = float(text[0].split(' ')[3])
    list_for_lines.pop()
    for i in list_for_lines:
        residuals = float(i[3])
        if residuals < 300000.0:
            z_list.append(float(i[0]))
            if len(i[1]) <= 12:
                x_list.append(i[1])
            else: 
                x_list.append(i[1][:-6])
            if len(i[2]) <= 12:
                y_list.append(i[2])
            else: 
                y_list.append(i[2][:-6])
            colour_list.append(float(i[3]))
        

In [7]:
df = pd.DataFrame({'ra': x_list, 'dec': y_list, 'freq': z_list, 'residuals': colour_list})
fig = px.scatter_3d(df, x='ra', y='dec', z='freq', color='residuals')
plotly.offline.plot(fig, filename=f'./res_iter_ra_dec_freq_{name_pulsar}.html')

'./res_iter_ra_dec_freq_1112.html'

### Чтение файла с координатами с частотой

In [11]:
PULSAR = '1112'


p_coors = pd.read_csv(f'res_iter_p_coords_{PULSAR}.txt',
    sep='\s+',
    names=['Freq, s-1', 'RAJ', 'DECJ', 'STD_res, us'],
    dtype={'Freq, s-1': np.float64, 'RAJ': str, 'DECJ': str, 'STD_res, us': np.float64}
)
pd.options.display.float_format = '{:.10f}'.format

In [12]:
p_coors_s = p_coors.sort_values(by='STD_res, us', ascending=True).reset_index(drop=True)

In [13]:
p_coors_s.head(20)

,"Freq, s-1",RAJ,DECJ,"STD_res, us"
0,0.6037000000,11:15:48.83478261,50:45:12.29,458078.0912565014
1,0.6037000000,11:15:48.83478261,50:45:36.61432432,458078.0971694730
2,0.6037000000,11:15:54.05217391,50:50:52.83054054,458344.2958693962
3,0.6037000000,11:15:43.6173913,50:45:36.61432432,458477.2621676719
4,0.6037000000,11:15:43.6173913,50:46:00.93864865,458477.2697963412
5,0.6037000000,11:15:59.26956522,50:51:17.15486486,458894.5006107469
6,0.6037000000,11:15:48.83478261,50:46:25.26297297,459005.3865234353
7,0.6037000000,11:15:48.83478261,50:46:49.5872973,459005.3907470123
8,0.6037000000,11:15:54.05217391,50:50:28.50621622,459026.6286850927
9,0.6037000000,11:15:48.83478261,50:46:00.93864865,459101.0830539749


## Код позволяющий выделить **ВСЕ** хорошие решения с разными частотами

In [14]:
best_sol = pd.DataFrame(columns=[
    'Freq, s-1', 
    'RAJ',
    'DECJ',
    'STD_res, us'
])
tres = 600000
idx = 0
for i in range(len(p_coors_s)):
    if p_coors_s['STD_res, us'][i] > tres:
        break
    if p_coors_s['Freq, s-1'][i] not in list(best_sol['Freq, s-1']):
        best_sol.loc[idx] = [
            np.float64(p_coors_s['Freq, s-1'][i]),
            p_coors_s['RAJ'][i],
            p_coors_s['DECJ'][i],
            p_coors_s['STD_res, us'][i]
        ]
        idx += 1

In [15]:
best_sol

,"Freq, s-1",RAJ,DECJ,"STD_res, us"
0,0.6037000000,11:15:48.83478261,50:45:12.29,458078.0912565014


In [76]:
z_list_1 = []
z_list_2 = []
z_list_3 = []
z_list_4 = []
z_list_5 = []
z_list_6 = []
z_list_7 = []
z_list_8 = []
z_list_9 = []
z_list_10 = []
for i in range(len(z_list)):
    if i <= 26999:
        z_list_1.append(z_list[i])
    elif 26999 < i <= 53999:
        z_list_2.append(z_list[i])
    elif 53999 < i <= 80999:
        z_list_3.append(z_list[i])
    elif 80999 < i <= 107999:
        z_list_4.append(z_list[i])
    elif 107999 < i <= 134999:
        z_list_5.append(z_list[i])
    elif 134999 < i <= 161999:
        z_list_6.append(z_list[i])
    elif 161999 < i <= 188999:
        z_list_7.append(z_list[i])
    elif 188999 < i <= 215999:
        z_list_8.append(z_list[i])
    elif 215999 < i <= 242999:
        z_list_9.append(z_list[i])
    elif 242999 < i <= 269999:
        z_list_10.append(z_list[i])

In [82]:
x_list_1 = []
x_list_2 = []
x_list_3 = []
x_list_4 = []
x_list_5 = []
x_list_6 = []
x_list_7 = []
x_list_8 = []
x_list_9 = []
x_list_10 = []
for i in range(len(x_list)):
    if i <= 26999:
        x_list_1.append(x_list[i])
    elif 26999 < i <= 53999:
        x_list_2.append(x_list[i])
    elif 53999 < i <= 80999:
        x_list_3.append(x_list[i])
    elif 80999 < i <= 107999:
        x_list_4.append(x_list[i])
    elif 107999 < i <= 134999:
        x_list_5.append(x_list[i])
    elif 134999 < i <= 161999:
        x_list_6.append(x_list[i])
    elif 161999 < i <= 188999:
        x_list_7.append(x_list[i])
    elif 188999 < i <= 215999:
        x_list_8.append(x_list[i])
    elif 215999 < i <= 242999:
        x_list_9.append(x_list[i])
    elif 242999 < i <= 269999:
        x_list_10.append(x_list[i])

In [83]:
y_list_1 = []
y_list_2 = []
y_list_3 = []
y_list_4 = []
y_list_5 = []
y_list_6 = []
y_list_7 = []
y_list_8 = []
y_list_9 = []
y_list_10 = []
for i in range(len(y_list)):
    if i <= 26999:
        y_list_1.append(y_list[i])
    elif 26999 < i <= 53999:
        y_list_2.append(y_list[i])
    elif 53999 < i <= 80999:
        y_list_3.append(y_list[i])
    elif 80999 < i <= 107999:
        y_list_4.append(y_list[i])
    elif 107999 < i <= 134999:
        y_list_5.append(y_list[i])
    elif 134999 < i <= 161999:
        y_list_6.append(y_list[i])
    elif 161999 < i <= 188999:
        y_list_7.append(y_list[i])
    elif 188999 < i <= 215999:
        y_list_8.append(y_list[i])
    elif 215999 < i <= 242999:
        y_list_9.append(y_list[i])
    elif 242999 < i <= 269999:
        y_list_10.append(y_list[i])

In [84]:
colour_list_1 = []
colour_list_2 = []
colour_list_3 = []
colour_list_4 = []
colour_list_5 = []
colour_list_6 = []
colour_list_7 = []
colour_list_8 = []
colour_list_9 = []
colour_list_10 = []
for i in range(len(colour_list)):
    if i <= 26999:
        colour_list_1.append(colour_list[i])
    elif 26999 < i <= 53999:
        colour_list_2.append(colour_list[i])
    elif 53999 < i <= 80999:
        colour_list_3.append(colour_list[i])
    elif 80999 < i <= 107999:
        colour_list_4.append(colour_list[i])
    elif 107999 < i <= 134999:
        colour_list_5.append(colour_list[i])
    elif 134999 < i <= 161999:
        colour_list_6.append(colour_list[i])
    elif 161999 < i <= 188999:
        colour_list_7.append(colour_list[i])
    elif 188999 < i <= 215999:
        colour_list_8.append(colour_list[i])
    elif 215999 < i <= 242999:
        colour_list_9.append(colour_list[i])
    elif 242999 < i <= 269999:
        colour_list_10.append(colour_list[i])

In [87]:
df_1 = pd.DataFrame({'ra': x_list_1, 'dec': y_list_1, 'freq': z_list_1, 'residuals': colour_list_1})
df_2 = pd.DataFrame({'ra': x_list_2, 'dec': y_list_2, 'freq': z_list_2, 'residuals': colour_list_2})
df_3 = pd.DataFrame({'ra': x_list_3, 'dec': y_list_3, 'freq': z_list_3, 'residuals': colour_list_3})
df_4 = pd.DataFrame({'ra': x_list_4, 'dec': y_list_4, 'freq': z_list_4, 'residuals': colour_list_4})
df_5 = pd.DataFrame({'ra': x_list_5, 'dec': y_list_5, 'freq': z_list_5, 'residuals': colour_list_5})
df_6 = pd.DataFrame({'ra': x_list_6, 'dec': y_list_6, 'freq': z_list_6, 'residuals': colour_list_6})
df_7 = pd.DataFrame({'ra': x_list_7, 'dec': y_list_7, 'freq': z_list_7, 'residuals': colour_list_7})
df_8 = pd.DataFrame({'ra': x_list_8, 'dec': y_list_8, 'freq': z_list_8, 'residuals': colour_list_8})
df_9 = pd.DataFrame({'ra': x_list_9, 'dec': y_list_9, 'freq': z_list_9, 'residuals': colour_list_9})
df_10 = pd.DataFrame({'ra': x_list_10, 'dec': y_list_10, 'freq': z_list_10, 'residuals': colour_list_10})

In [94]:
fig_1 = px.scatter_3d(df_1, x='ra', y='dec', z='freq', color='residuals')
fig_2 = px.scatter_3d(df_2, x='ra', y='dec', z='freq', color='residuals')
fig_3 = px.scatter_3d(df_3, x='ra', y='dec', z='freq', color='residuals')
fig_4 = px.scatter_3d(df_4, x='ra', y='dec', z='freq', color='residuals')
fig_5 = px.scatter_3d(df_5, x='ra', y='dec', z='freq', color='residuals')
fig_6 = px.scatter_3d(df_6, x='ra', y='dec', z='freq', color='residuals')
fig_7 = px.scatter_3d(df_7, x='ra', y='dec', z='freq', color='residuals')
fig_8 = px.scatter_3d(df_8, x='ra', y='dec', z='freq', color='residuals')
fig_9 = px.scatter_3d(df_9, x='ra', y='dec', z='freq', color='residuals')
fig_10 = px.scatter_3d(df_10, x='ra', y='dec', z='freq', color='residuals')

In [146]:
Min_val_freq = int(input('Введите минимальный номер слоя: '))
Max_val_freq = int(input('Введите максимальный номер слоя: '))
result_x_list, result_y_list, result_z_list, result_colour_list = [], [], [], []
x_list, y_list, z_list, colour_list = [x_list_1, x_list_2, x_list_3, 
                                       x_list_4, x_list_5, x_list_6, x_list_7, 
                                       x_list_8, x_list_9, x_list_10], [y_list_1, y_list_2, y_list_3, y_list_4, 
                                                                        
                                                                        y_list_5, y_list_6, y_list_7, y_list_8, 
                                                                        y_list_9, y_list_10], [z_list_1, z_list_2, z_list_3, z_list_4, 
                                                                                               z_list_5, z_list_6, z_list_7, 
                                                                                               z_list_8, z_list_9, z_list_10], [colour_list_1, colour_list_2, colour_list_3, colour_list_4, 
                                                                                                                                colour_list_5, colour_list_6, colour_list_7, colour_list_8, colour_list_9, colour_list_10]
for i in range(Min_val_freq, Max_val_freq+1):
    result_x_list.extend(x_list[i-1])
    result_y_list.extend(y_list[i-1])
    result_z_list.extend(z_list[i-1])
    result_colour_list.extend(colour_list[i-1])


df = pd.DataFrame({'ra': result_x_list, 'dec': result_y_list, 'freq': result_z_list, 'residuals': result_colour_list})
fig = px.scatter_3d(df, x='ra', y='dec', z='freq', color='residuals')
plotly.offline.plot(fig, filename=f'./res_iter_ra_dec_freq_{name_pulsar}.html')

Введите минимальный номер слоя:  5
Введите максимальный номер слоя:  5


'./res_iter_ra_dec_freq_1112.html'

## 